In [ ]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import (
    RandomForestRegressor,
    HistGradientBoostingRegressor,
    RandomForestClassifier,
    HistGradientBoostingClassifier
)

from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    train_test_split,
    cross_validate,
    cross_val_score
)

from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, LogisticRegression

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    f1_score,
    roc_auc_score,
    average_precision_score
)

import matplotlib.pyplot as plt
import joblib
import os


In [ ]:
main_df = pd.read_csv('../data/processed/datasets_merged_cleaned.csv')

main_df['fecha'] = pd.to_datetime(main_df['anio'].astype(str) + '-' + main_df['mes'].astype(str), format='%Y-%m')
unique_cadenas = set([(a,b) for a,b in main_df[['cadena_id', 'cadena']].values])

# Guarda en disco el modelo en formato joblib para su posterior uso.
def guardar_modelo(modelo, filename):
    path = os.path.join('../models', filename)
    
    try: os.makedirs(os.path.dirname(path))
    except: pass

    joblib.dump(modelo, path)

# carga un modelo desde disco
def cargar_modelo(filename):
    path = os.path.join('../models', filename)
    
    if os.path.exists(path): return joblib.load(path)
    return None

REGRESION

In [ ]:
calcular_rmse = lambda y_test, y_pred: np.sqrt(mean_squared_error(y_test, y_pred))
calcular_mae = lambda y_test, y_pred: mean_absolute_error(y_test, y_pred)
calcular_wmape = lambda y_test, y_pred: np.sum(np.abs(y_test - y_pred)) / np.sum(np.abs(y_test))

# Iterador, genera una tupla (cadena, X, y).
def iter_regresion_Xy():
	# Las regresiones se harán en grupos separados por cadena de supermercados, ya que se quiere evitar
	# la influencia de los precios de una cadena sobre los de otra. Usualmente una misma cadena oscila sus
	# precios de forma regular.
	for cadena_id,cadena_name in unique_cadenas:
		df = main_df[main_df.cadena_id == cadena_id]
		
		# Ya fueron codificadas previamente las variables categóricas, por lo que se utilizan sus ids directamente.
		X = df[['supermercado_id', 'cadena_id', 'producto_id']]
		y = df['costo']
		
		yield cadena_name, X, y

# Listado de todos los modelos de regresión que se utilizarán para el análisis.
all_regresion = {
    'Baseline': DummyRegressor(strategy='mean'),
    'Ridge': Ridge(alpha=1.0, max_iter=10000),
    'RandomForest': RandomForestRegressor(random_state=42),
    'HistGradientBoosting': HistGradientBoostingRegressor(random_state=42),
}

metricas = []

for cadena_name, X, y in iter_regresion_Xy():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for reg_nombre, reg in all_regresion.items():
        modelo = Pipeline([
            ('scaler', StandardScaler()),
            ('regresion', reg),
        ])
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
                
        guardar_modelo(modelo, 'regresion/{} - {}.joblib'.format(cadena_name, reg_nombre))
        
		# Cálculo de las métricas MAE, RMSE y WMAPE
        mae = calcular_mae(y_test, y_pred)
        rmse = calcular_rmse(y_test, y_pred)
        wmape = calcular_wmape(y_test, y_pred)

        metricas.append({
            'cadena': cadena_name,
            'regresion': reg_nombre,
            'MAE': mae,
            'RMSE': rmse,
            'WMAPE': wmape,
        })
    
metricas = pd.DataFrame(metricas)
display(metricas)
    

REGRESION - VALIDACION CRUZADA

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
regresion_scores = pd.DataFrame()

# Utiliza el iterador para volver a obtener (cadena, X, y).
for cadena_name, X, y in iter_regresion_Xy():
    puntajes = []
    
	# Se realizará una validación cruzada de los modelos de regresión que se seleccionaron.
    for reg_nombre, reg in all_regresion.items():
        modelo = Pipeline([
            ('scaler', StandardScaler()),
            ('regresion', reg),
        ])
        
        scores = cross_val_score(modelo, X, y, cv=kf, scoring='neg_mean_absolute_error')
        
        puntajes.append({
            'cadena': cadena_name,
            'regresion': reg_nombre,
            'MAE score': -np.mean(scores)
        })
    
	# Se utiliza la métrica MAE para evaluar el mejor modelo.
	# El mejor puntaje es aquel cuya MAE sea menor.
    puntajes = pd.DataFrame(puntajes)
    puntajes['mejor'] = puntajes['MAE score'] == puntajes['MAE score'].min()
    regresion_scores = pd.concat([regresion_scores, puntajes])

display(regresion_scores)        

In [ ]:
#Graficar los mejores modelos de regresión para cada cadena.
mejores_modelos = regresion_scores[regresion_scores.mejor == True][['cadena', 'regresion']]

for cadena, regresion in mejores_modelos.values:
    # Se carga el modelo previamente guardado en disco
    modelo = cargar_modelo('regresion/{} - {}.joblib'.format(cadena, regresion))
    
    if modelo:
        df = main_df[main_df.cadena == cadena]
        y_pred = modelo.predict(df[['supermercado_id', 'cadena_id', 'producto_id']])

        fig, ax = plt.subplots(1, 1, figsize=(10, 5))
        ax.set_title('Estimaciones de costos para cadena de supermercados: %s' % cadena, fontsize=10)
        ax.tick_params(axis='x', labelsize=6)
        ax.tick_params(axis='y', labelsize=6)
        ax.grid(True, color='#dedede')
        
        for x,y,label in [(df.fecha, df.costo, 'real'), (df.fecha, y_pred, 'regresión')]:
            df = pd.DataFrame({'fecha':x, 'costo':y}).groupby(['fecha'])['costo'].mean().reset_index()
            ax.plot(df.fecha, df.costo, label=label)

    plt.legend()
    plt.show()

CLASIFICACION

In [ ]:
# Similar a la regresión, se crea un iterador que genera una tupla (cadena, X, y).
def iter_clasificacion_Xy():
    for cadena_id, cadena_name in unique_cadenas:
        df = main_df[main_df.cadena_id == cadena_id][['supermercado_id', 'cadena_id', 'producto_id', 'costo', 'fecha']]
        df = df.sort_values('fecha')

        umbral = df['costo'].median()

        df['target'] = (umbral < df['costo']).astype(int)
        X = df[['supermercado_id', 'cadena_id', 'producto_id']]
        y = df['target']

        yield cadena_name, X, y

# Se enlistan los modelos de clasificación que se utilizarán en el análisis.
all_clasificacion = {
    'Baseline': DummyClassifier(strategy="stratified", random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42),
    'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42)
}

metricas = []

for cadena_name, X, y in iter_clasificacion_Xy():
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    
    for cls_nombre, cls in all_clasificacion.items():
        modelo = Pipeline([
            ('scaler', StandardScaler()),
            ('clasificador', cls),
        ])
        
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        
        # Se guarda el modelo en disco.
        guardar_modelo(modelo, 'clasificacion/{} - {}.joblib'.format(cadena_name, cls_nombre))
        
        # Algunos modelos generan probabilidades, por lo que hay que verificar cuáles lo hacen.
        if hasattr(modelo.named_steps["clasificador"], "predict_proba"):
            y_proba = modelo.predict_proba(X_test)[:, 1]
        elif hasattr(modelo.named_steps["clasificador"], "decision_function"):
            y_proba = modelo.decision_function(X_test)
        else:
            y_proba = None

        f1 = f1_score(y_test, y_pred, average='macro')
        roc_auc = None if y_proba is None else roc_auc_score(y_test, y_proba)
        pr_auc = None if y_proba is None else average_precision_score(y_test, y_proba)
        
        metricas.append({
            'cadena': cadena_name,
            'clasificador': cls_nombre,
            'F1': f1,
            'ROC-AUC': roc_auc,
            'PR-AUC': pr_auc,
        })

metricas = pd.DataFrame(metricas)
display(metricas)

CLASIFICACION - VALIDACION CRUZADA

In [ ]:
# Validación cruzada para los modelos de clasificación.

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clasificacion_scores = pd.DataFrame()

# Los puntajes estarán basados en las métricas F1 obligatorio, ROC-AUC y PR-AUC
scoring = {
    'f1_macro': 'f1_macro',
    'roc_auc': 'roc_auc',
    'pr_auc': 'average_precision'  # PR-AUC
}

for cadena_name, X, y in iter_clasificacion_Xy():
    puntajes = []
    
    for cls_nombre, cls in all_clasificacion.items():
        scores = cross_validate(cls, X, y, cv=cv, scoring=scoring, return_train_score=False)
        
        puntajes.append({
            'cadena': cadena_name,
            'clasificador': cls_nombre,
            'F1 score': np.mean(scores['test_f1_macro']),
            'ROC-AUC score': np.mean(scores['test_roc_auc']),
            'PR-AUC score': np.mean(scores['test_pr_auc'])
        })
    
	# Se utiliza F1 como métrica para determinar el mejor modelo. Esto debido a que el objetivo de la clasificación
    # es determinar si el costo subirá o no de valor al siguiente mes. Para dicho análisis, F1 es la mejor opción.
    # Mientras más alto el valor F1, mejor.
    puntajes = pd.DataFrame(puntajes)
    puntajes['mejor'] = puntajes['F1 score'] == puntajes['F1 score'].max()
    clasificacion_scores = pd.concat([clasificacion_scores, puntajes])

display(clasificacion_scores)